# Explore here

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
zeros_count = (df == 0).sum()
zeros_count

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop("Outcome", axis = 1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI

In [8]:
import numpy as np
from sklearn.impute import SimpleImputer
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
X_train[columns] = X_train[columns].replace(0, np.nan)
X_test[columns] = X_test[columns].replace(0, np.nan)
imputer = SimpleImputer(strategy="mean")
X_train[columns] = imputer.fit_transform(X_train[columns])
X_test[columns] = imputer.transform(X_test[columns])


In [9]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state= 42)
print(model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],         # Number of trees
    'max_depth': [10, 20, None],           # Tree depth
    'min_samples_split': [2, 5, 10],       # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],         # Minimum samples in a leaf
    'max_features': ['sqrt', 'log2'],      # Number of features considered for splits
    'bootstrap': [True, False]             # Whether to bootstrap samples
}
# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',     # Optimize for accuracy
    cv=5,                   # 5-fold cross-validation
    verbose=1,              # Show progress
    n_jobs=-1               # Use all available CPU cores
)
# Perform the grid search
grid_search.fit(X_train, y_train)
# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best cross-validation score: 0.7899373583899774


In [11]:
# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", test_accuracy)

Test accuracy: 0.7467532467532467
